In [1]:
library(repr)
options(repr.plot.width = 15, repr.plot.height = 10)

In [2]:
#| include: false

library(here)
library(IRdisplay)
source(here("R", "_library.r"))
source(here("R", "_outcome_labels.r"))

root <- "D:/JMDPフィールド実験"

In [ ]:
#| include: false

rawdt <- read_csv(here(root, "shaped.csv"), locale = locale(encoding = "cp932"))

use <- rawdt %>%
  dplyr::filter(ongoing == 0) %>%
  mutate(
    treat = factor(treat, levels = LETTERS[1:4]),
    plan_two_methods = if_else(plan_method == "BM/PB", 1, 0)
  )

stock <- use %>%
  dplyr::filter(exg_stop_reply == 0) %>%
  rename(positive = intention) %>%
  mutate(
    negative = reply * (1 - positive),
    age_demean = age - mean(rawdt$age),
  ) %>%
  select(reply, positive, negative, everything()) %>%
  pivot_longer(reply:negative, "outcome") %>%
  mutate(outcome = factor(
    outcome,
    levels = unlist(names(outcome_label)[1:3]),
    labels = unlist(outcome_label[1:3])
  ))

In [20]:
tidy_custom.glm <- function(x, ...) {
  tbl <- tidy(x) %>%
    mutate(
      or = exp(estimate),
      lower.or = exp(estimate - 1.96 * std.error),
      upper.or = exp(estimate + 1.96 * std.error)
    )

  tbl
}

In [31]:
mod <- list(
  unctrl = value ~ treat,
  ctrl = value ~ treat + age_demean + I(age_demean^2) + male + coordinate +
    hospital_per_area + PB_per_area + BM_per_area +
    factor(prefecture) + factor(month) + factor(week)
)

est_stock <- stock %>%
  group_by(outcome) %>%
  nest() %>%
  mutate(
    fit1 = map(
      data,
      ~ glm(
        mod$unctrl,
        data = .,
        family = binomial()
      )
    ),
    fit2 = map(
      data,
      ~ glm(
        mod$ctrl,
        data = .,
        family = binomial()
      )
    )
  ) %>%
  pivot_longer(
    fit1:fit2,
    names_prefix = "fit",
    names_to = "model",
    values_to = "fit"
  )

add_tables <- tibble::tribble(
  ~term, ~"(1)", ~"(2)", ~"(3)", ~"(4)", ~"(5)", ~"(6)",
  "Covariates", "", "X", "", "X", "", "X"
)

attr(add_tables, "position") <- 7

est_stock %>%
  pull(fit) %>%
  setNames(paste0("(", seq_len(length(.)), ")")) %>%
  modelsummary(
    title = "Logit Model of Reply and Intention",
    estimate = "{or}",
    statistic = "[{lower.or}, {upper.or}]",
    coef_map = c(
      "treatB" = "Treatment B",
      "treatC" = "Treatment C",
      "treatD" = "Treatment D"
    ),
    add_rows = add_tables,
    gof_omit = "R2|AIC|BIC|RMSE|Std|FE|se_type",
    fmt = fmt_sprintf("%.3f")
  ) %>%
  kableExtra::kable_styling() %>%
  kableExtra::add_header_above(c(
    " " = 1, "Reply" = 2, "Positive" = 2, "Negative" = 2
  )) %>%
  kableExtra::add_header_above(c(
    " " = 3, "Intention" = 4
  )) %>%
  kableExtra::footnote(
    paste(
      "We report odds ratio and its 95% confidential interval.",
      "Covariates are gender, squared polynomial of (demeaned) age,",
      "number of past coordinations,",
      "number of hospitals per 10 square kilometers,",
      "number of hospitals with PBSC collection per 10 square kilometers,",
      "number of hospitals with BM collection per 10 square kilometers,",
      "prefecture dummies, month dummies, and week dummies."
    )
  ) %>%
  as.character() %>%
  display_html()

Warning message in predict.lm(object, newdata, se.fit, scale = 1, type = if (type == :
"prediction from a rank-deficient fit may be misleading"
Warning message in predict.lm(object, newdata, se.fit, scale = 1, type = if (type == :
"prediction from a rank-deficient fit may be misleading"
Warning message in predict.lm(object, newdata, se.fit, scale = 1, type = if (type == :
"prediction from a rank-deficient fit may be misleading"


Logit Model of Reply and Intention 
 
 
 
 Intention 
 
 
 
 Reply 
 Positive 
 Negative 
 
 
 
  (1) 
   (2) 
   (3) 
   (4) 
   (5) 
   (6) 
 
 
 
 
 Treatment B 
 1.112 
 1.143 
 1.093 
 1.091 
 0.950 
 0.967 
 
 
 
 [0.940, 1.315] 
 [0.958, 1.364] 
 [0.982, 1.216] 
 [0.975, 1.221] 
 [0.848, 1.063] 
 [0.860, 1.088] 
 
 
 Treatment C 
 0.947 
 1.003 
 0.979 
 0.978 
 0.998 
 1.028 
 
 
 
 [0.801, 1.120] 
 [0.837, 1.201] 
 [0.878, 1.092] 
 [0.869, 1.100] 
 [0.889, 1.120] 
 [0.909, 1.162] 
 
 
 Treatment D 
 1.060 
 1.062 
 1.019 
 1.027 
 1.005 
 0.999 
 
 
 
 [0.893, 1.257] 
 [0.886, 1.272] 
 [0.914, 1.136] 
 [0.915, 1.153] 
 [0.896, 1.128] 
 [0.886, 1.127] 
 
 
 Covariates 
 
 X 
 
 X 
 
 X 
 
 
 Num.Obs. 
 10985 
 10985 
 10985 
 10985 
 10985 
 10985 
 
 
 Log.Lik. 
 -3884.517 
 -3629.432 
 -7534.803 
 -7254.947 
 -6945.023 
 -6818.753 
 
 
 
 Note: 
 
 We report odds ratio and its 95% confidential interval. Covariates are gender, squared polynomial of (demeaned) age, number of past coordinations, number of hospitals per 10 square kilometers, number of hospitals with PBSC collection per 10 square kilometers, number of hospitals with BM collection per 10 square kilometers, prefecture dummies, month dummies, and week dummies.